In [2]:
%reload_ext autoreload
%autoreload 2

from datasets import load_dataset
from prompt import math_word_problem_template
import utils

In [3]:
ds_main = load_dataset('gsm8k', 'main', split='test')
ds_main[0]

[2023-01-16 07:50:10,007] [datasets.builder] [builder.py:785] Found cached dataset gsm8k (/workspaces/seed/cache/hf_dataset/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [29]:
len(ds_main)

1319

In [4]:
ds_socratic = load_dataset('gsm8k', 'socratic', split='test')
ds_socratic[0]

[2023-01-16 07:50:14,331] [datasets.builder] [builder.py:785] Found cached dataset gsm8k (/workspaces/seed/cache/hf_dataset/gsm8k/socratic/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': "How many eggs does Janet sell? ** Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nHow much does Janet make at the farmers' market? ** She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18"}

`main` subset lays out the rationale objectively.  
`socratic` subset carries on a series of self qa subjectively. 

Prompt in the paper is more akin to `main` style. 

In [5]:
checkpoint='google/flan-t5-xl'

t2t = utils.build_t2t(checkpoint)

[2023-01-16 07:50:14,401] [CoT] [utils.py:15] loading model from google/flan-t5-xl...


In [19]:
t0 = math_word_problem_template(ds_main[0])
print(t0)

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today? 
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6. 

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot? 
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5. 

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total? 
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39. 

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny? 
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave De

In [20]:
pred0 = t2t(t0)[0]
print(pred0)

She eats 3 + 4 = 7 eggs every day. She has 16 - 7 = 9 eggs left. She sells 9 * 2 = $18 at the farmers' market. The answer is 18.


When `num_beams=4`, answer = **Janet eats 3 + 4 = 7 eggs a day. She sells 16 - 7 = 11 eggs a day at the farmers' market. She makes 11 * 2 = 22 dollars a day at the farmers' market. The answer is 22.**

Even though the answer is wrong. The miss step is 16-7=9, not 11. The logic overall is fine. This is the emergent part of the LLM for reasoning task. So many little intermediate step could go wrong, and on wrong would render the final answer wrong.   

It's actually not emergent. To get one reasoning task right, observed from final answer perspective, it's a quantum leap wrt to modele size. However, observed from rationale perpsective, it's not quantum leap and magical anymore. You can observe little mistakes here and there, and they lead to wrong answer. 

The other mistake is Janet won't eat 7 eggs, but rather use 7 per day. Semantically, this 3b model got it wrong, even though the number, 7, is the right number for next stage calculation. 

Also you can see the beauty of such thinking out loud rationale engineering. It makes abstract computation transparent and **readable** so you could understand and debug incrementally, rather than betting on pure wishful thinking of quantum leap. 

When I switch off `num_beams` to `greedy decoding`, it actually generate right answer at 3b scale.

In [21]:
a0 = ds_main[0]['answer']
print(a0)

Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [22]:
utils.regex_target(a0) == utils.regex_predict(pred0)

True

In [26]:
t1 = math_word_problem_template(ds_main[1])
pred1 = t2t(t1)[0]
pred1

'The robe takes 2 / 2 = 1 bolt of white fiber. So it takes 2 + 1 = 3 bolts of fiber in total. The answer is 3.'

In [27]:
a1 = ds_main[1]['answer']
print(a1)

It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3


In [28]:
utils.regex_target(a1) == utils.regex_predict(pred1)

True